<a href="https://colab.research.google.com/github/nimrashaheen001/Programming_for_AI/blob/main/MMGFL(LAST_TRY).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install SimpleITK
import SimpleITK as sitk
import torch
from torch.utils.data.dataset import Dataset
import os
import csv
import numpy as np
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define base path for your data in Google Drive
BASE_PATH = '/content/drive/My Drive/pythonProject9'

def resize_image(image, new_size, resamplemethod=sitk.sitkLinear):
    """Resize 3D image to new size."""
    dimension = image.GetDimension()
    reference_physical_size = np.zeros(dimension)
    reference_physical_size[:] = [(sz-1)*spc if sz*spc>mx else mx for sz, spc, mx in zip(new_size, image.GetSpacing(), image.GetSize())]
    reference_origin = image.GetOrigin()
    reference_direction = image.GetDirection()
    reference_size = new_size
    reference_spacing = [ phys_sz/(sz-1) for sz, phys_sz in zip(reference_size, reference_physical_size) ]
    reference_image = sitk.Image(reference_size, image.GetPixelIDValue())
    reference_image.SetOrigin(reference_origin)
    reference_image.SetSpacing(reference_spacing)
    reference_image.SetDirection(reference_direction)
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(resamplemethod)
    resample.SetOutputSpacing(reference_spacing)
    resample.SetSize(reference_size)
    resample.SetOutputOrigin(reference_origin)
    resample.SetOutputDirection(reference_direction)
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(image.GetPixelIDValue())
    return resample.Execute(image)

def norm_image(image):
    """Normalize image tensor."""
    mean = torch.mean(image)
    std = torch.std(image)
    image = (image - mean) / std
    return image

class ADMdataset(Dataset):
    def __init__(self, data_txt):
        """
        Initialize dataset.
        data_txt: path to text file containing image paths and labels
        """
        self.data_txt = os.path.join(BASE_PATH, data_txt)
        self.datasets = []

        # Read the text file
        with open(self.data_txt, 'r') as f:
            for line in f:
                image_file = line.strip('\n').split(' ')[0]
                image_label = line.strip('\n').split(' ')[1]
                # Convert relative paths to absolute Google Drive paths
                full_image_path = os.path.join(BASE_PATH, image_file)
                self.datasets.append([full_image_path, image_label])

        print(f"Loaded {len(self.datasets)} samples from {data_txt}")

    def __getitem__(self, idx):
        """Get a single sample from the dataset."""
        try:
            # Get image path and create paths
            image_path = self.datasets[idx][0]
            dir_name = os.path.dirname(os.path.dirname(os.path.dirname(image_path)))
            txt_file_path = os.path.join(dir_name, 'tabular.csv')

            # Load and process MRI images
            series_reader = sitk.ImageSeriesReader()
            fileNames = series_reader.GetGDCMSeriesFileNames(image_path)

            if not fileNames:
                raise RuntimeError(f"No DICOM series found in {image_path}")

            series_reader.SetFileNames(fileNames)
            images = series_reader.Execute()

            # Resize images
            images = resize_image(images, (64, 64, 64), resamplemethod=sitk.sitkLinear)

            # Convert to tensor
            img_array = sitk.GetArrayFromImage(images)
            img_vol = torch.from_numpy(img_array)
            img_vol = norm_image(img_vol)
            img_vol = img_vol.unsqueeze(0).float()

            # Process label
            image_label = float(self.datasets[idx][1])

            # Get patient ID
            patient_id = os.path.basename(image_path)

            # Load tabular data
            tabular_data = []
            tabular_file = os.path.join(BASE_PATH, txt_file_path)

            with open(tabular_file, 'r') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    if row['PTID'] == patient_id:
                        row_data = []
                        for key, value in row.items():
                            if key != 'PTID':
                                try:
                                    row_data.append(float(value))
                                except ValueError:
                                    print(f"Warning: Could not convert value {value} to float")
                                    row_data.append(0.0)
                        tabular_data = torch.tensor(row_data)
                        break

            if not len(tabular_data):
                raise RuntimeError(f"No tabular data found for patient {patient_id}")

            return img_vol, torch.tensor(image_label), tabular_data

        except Exception as e:
            print(f"Error loading sample {idx}: {str(e)}")
            # Return a dummy sample in case of error
            return torch.zeros((1, 64, 64, 64)), torch.tensor(0.0), torch.zeros(10)

    def __len__(self):
        """Return the total number of samples."""
        return len(self.datasets)

# Example usage
def test_dataset():
    """Test the dataset loader"""
    try:
        # Create dataset instance
        dataset = ADMdataset('train-oasis.txt')

        print(f"\nDataset size: {len(dataset)}")

        # Try loading first sample
        img, label, tabular = dataset[0]

        print("\nSample information:")
        print(f"MRI shape: {img.shape}")
        print(f"Label: {label}")
        print(f"Tabular data shape: {tabular.shape}")

        # Create DataLoader
        from torch.utils.data import DataLoader
        loader = DataLoader(dataset, batch_size=4, shuffle=True)

        # Try loading a batch
        for batch_idx, (images, labels, tabular) in enumerate(loader):
            print(f"\nBatch {batch_idx + 1}:")
            print(f"Batch image shape: {images.shape}")
            print(f"Batch labels shape: {labels.shape}")
            print(f"Batch tabular shape: {tabular.shape}")
            break  # Just test first batch

    except Exception as e:
        print(f"Error testing dataset: {str(e)}")

if __name__ == "__main__":
    # Mount Google Drive and test dataset
    drive.mount('/content/drive')
    test_dataset()

import torch
import torch.nn as nn
import torch.nn.functional as F
from collections.abc import Sequence  # Fixed import issue

# Helper function for 3D convolution
def conv3d(in_channels, out_channels, kernel_size=3, stride=1):
    padding = 1 if kernel_size != 1 else 0
    return nn.Conv3d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=False)

# 3D Convolution + BatchNorm + ReLU Block
class ConvBnReLU(nn.Module):
    def __init__(self, in_channels, out_channels, bn_momentum=0.05, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = nn.BatchNorm3d(out_channels, momentum=bn_momentum)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

# Residual Block
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, bn_momentum=0.05, stride=1):
        super().__init__()
        self.conv1 = conv3d(in_channels, out_channels, stride=stride)
        self.bn1 = nn.BatchNorm3d(out_channels, momentum=bn_momentum)
        self.conv2 = conv3d(out_channels, out_channels)
        self.bn2 = nn.BatchNorm3d(out_channels, momentum=bn_momentum)
        self.relu = nn.ReLU(inplace=True)

        self.downsample = (
            nn.Sequential(conv3d(in_channels, out_channels, kernel_size=1, stride=stride),
                          nn.BatchNorm3d(out_channels, momentum=bn_momentum))
            if stride != 1 or in_channels != out_channels else None
        )

    def forward(self, x):
        residual = x if self.downsample is None else self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        return self.relu(out + residual)

# Attention Mechanism
class Attention(nn.Module):
    def __init__(self, dim, heads=8, qkv_bias=False, qk_scale=None, dropout_rate=0.0):
        super().__init__()
        self.num_heads = heads
        head_dim = dim // heads
        self.scale = qk_scale or head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(dropout_rate)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(dropout_rate)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = self.attn_drop(attn.softmax(dim=-1))
        x = self.proj((attn @ v).transpose(1, 2).reshape(B, N, C))
        return self.proj_drop(x)

from _py_abc import ABCMeta
from typing import Optional, Dict, Any

import torch
import torch.nn as nn
import torch.nn.functional as F
from collections.abc import Sequence

from collections import OrderedDict
from typing import Any, Dict, Optional, Sequence

import torch
import torch.nn as nn

#from networks.blocks import ConvBnReLU, ResBlock, conv3d, Attention


class ConcatHNN1FC(nn.Module):
    def __init__(self, in_channels=1, n_outputs=3, bn_momentum=0.1, n_basefilters=4, ndim_non_img=10) -> None:
        super().__init__()
        self.conv1 = ConvBnReLU(in_channels, n_basefilters, bn_momentum=bn_momentum)
        self.pool1 = nn.MaxPool3d(2, stride=2)  # 32
        self.block1 = ResBlock(n_basefilters, n_basefilters, bn_momentum=bn_momentum)
        self.block2 = ResBlock(n_basefilters, 2 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 16
        self.block3 = ResBlock(2 * n_basefilters, 4 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 8
        self.block4 = ResBlock(4 * n_basefilters, 8 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 4
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(8 * n_basefilters + ndim_non_img, 32)
        self.fc1 = nn.Linear(32, n_outputs)


    def forward(self, image, tabular):
        out = self.conv1(image)
        out = self.pool1(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.global_pool(out)
        out = out.view(out.size(0), -1)
        out = torch.cat((out, tabular), dim=1)
        out = self.fc(out)
        out = self.fc1(out)
        return out


class ConcatHNN2FC(nn.Module):
    def __init__(self, in_channels=1, n_outputs=3, bn_momentum=0.1, n_basefilters=4, ndim_non_img=10, bottleneck_dim=15) -> None:
        super().__init__()
        self.conv1 = ConvBnReLU(in_channels, n_basefilters, bn_momentum=bn_momentum)
        self.pool1 = nn.MaxPool3d(2, stride=2)  # 32
        self.block1 = ResBlock(n_basefilters, n_basefilters, bn_momentum=bn_momentum)
        self.block2 = ResBlock(n_basefilters, 2 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 16
        self.block3 = ResBlock(2 * n_basefilters, 4 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 8
        self.block4 = ResBlock(4 * n_basefilters, 8 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 4
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(8 * n_basefilters + ndim_non_img, bottleneck_dim)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(bottleneck_dim, n_outputs)

    def forward(self, image, tabular):
        out = self.conv1(image)
        out = self.pool1(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.global_pool(out)
        out = out.view(out.size(0), -1)
        out = torch.cat((out, tabular), dim=1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)

        return out


class HeterogeneousResNet(nn.Module):
    def __init__(self, in_channels=1, n_outputs=3, bn_momentum=0.1, n_basefilters=4) -> None:
        super().__init__()

        self.conv1 = ConvBnReLU(in_channels, n_basefilters, bn_momentum=bn_momentum)
        self.pool1 = nn.MaxPool3d(2, stride=2)  # 32
        self.block1 = ResBlock(n_basefilters, n_basefilters, bn_momentum=bn_momentum)
        self.block2 = ResBlock(n_basefilters, 2 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 16
        self.block3 = ResBlock(2 * n_basefilters, 4 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 8
        self.block4 = ResBlock(4 * n_basefilters, 8 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 4
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(8 * n_basefilters, n_outputs)


    def forward(self, image):
        out = self.conv1(image)
        out = self.pool1(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.global_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out


class InteractiveHNN(nn.Module):

    def __init__(self, in_channels=1, n_outputs=3, bn_momentum=0.1, n_basefilters=4, ndim_non_img=10) -> None:
        super().__init__()

        # ResNet
        self.conv1 = ConvBnReLU(in_channels, n_basefilters, bn_momentum=bn_momentum)
        self.pool1 = nn.MaxPool3d(2, stride=2)  # 32
        self.block1 = ResBlock(n_basefilters, n_basefilters, bn_momentum=bn_momentum)
        self.block2 = ResBlock(n_basefilters, 2 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 16
        self.block3 = ResBlock(2 * n_basefilters, 4 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 8
        self.block4 = ResBlock(4 * n_basefilters, 8 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 4
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(8 * n_basefilters, n_outputs)

        layers = [
            ("aux_base", nn.Linear(ndim_non_img, 15, bias=False)),
            ("aux_relu", nn.ReLU()),
            # ("aux_dropout", nn.Dropout(p=0.2, inplace=True)),
            ("aux_1", nn.Linear(15, n_basefilters, bias=False)),
        ]
        self.aux = nn.Sequential(OrderedDict(layers))

        self.aux_2 = nn.Linear(n_basefilters, n_basefilters, bias=False)
        self.aux_3 = nn.Linear(n_basefilters, 2 * n_basefilters, bias=False)
        self.aux_4 = nn.Linear(2 * n_basefilters, 4 * n_basefilters, bias=False)


    def forward(self, image, tabular):
        out = self.conv1(image)
        out = self.pool1(out)
        tabular = tabular.to(torch.float32)

        attention = self.aux(tabular)
        batch_size, n_channels = out.size()[:2]
        out = torch.mul(out, attention.view(batch_size, n_channels, 1, 1, 1))
        out = self.block1(out)

        attention = self.aux_2(attention)
        batch_size, n_channels = out.size()[:2]
        out = torch.mul(out, attention.view(batch_size, n_channels, 1, 1, 1))
        out = self.block2(out)

        attention = self.aux_3(attention)
        batch_size, n_channels = out.size()[:2]
        out = torch.mul(out, attention.view(batch_size, n_channels, 1, 1, 1))
        out = self.block3(out)

        attention = self.aux_4(attention)
        batch_size, n_channels = out.size()[:2]
        out = torch.mul(out, attention.view(batch_size, n_channels, 1, 1, 1))
        out = self.block4(out)

        out = self.global_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out1 = F.softmax(out, dim=1)

        return out1

class DAFT2021(nn.Module):
    def __init__(
            self,
            in_channels: int=1,
            n_outputs: int=3,
            bn_momentum: float = 0.1,
            n_basefilters: int = 4,
            filmblock_args: Optional[Dict[Any, Any]] = None,
    ) -> None:
        super().__init__()

        if filmblock_args is None:
            filmblock_args = {}

        if filmblock_args is None:
            filmblock_args = {}

        self.split_size = 4 * n_basefilters
        self.conv1 = ConvBnReLU(in_channels, n_basefilters, bn_momentum=bn_momentum)
        self.pool1 = nn.MaxPool3d(2, stride=2)  # 32
        self.block1 = ResBlock(n_basefilters, n_basefilters, bn_momentum=bn_momentum)
        self.block2 = ResBlock(n_basefilters, 2 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 16
        self.block3 = ResBlock(2 * n_basefilters, 4 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 8
        self.blockX = DAFTBlock(4 * n_basefilters, 8 * n_basefilters, bn_momentum=bn_momentum, **filmblock_args)  # 4
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(8 * n_basefilters, n_outputs)



    def forward(self, image, tabular):
        out = self.conv1(image)
        out = self.pool1(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.blockX(out, tabular)
        out = self.global_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out


class FilmBase(nn.Module, metaclass=ABCMeta):

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bn_momentum: float,
        stride: int,
        ndim_non_img: int,
        location: int,
        activation: str,
        scale: bool,
        shift: bool,
    ) -> None:

        super().__init__()

        # sanity checks
        if location not in set(range(5)):
            raise ValueError(f"Invalid location specified: {location}")
        if activation not in {"tanh", "sigmoid", "linear"}:
            raise ValueError(f"Invalid location specified: {location}")
        if (not isinstance(scale, bool) or not isinstance(shift, bool)) or (not scale and not shift):
            raise ValueError(
                f"scale and shift must be of type bool:\n    -> scale value: {scale}, "
                "scale type {type(scale)}\n    -> shift value: {shift}, shift type: {type(shift)}"
            )
        # ResBlock
        self.conv1 = conv3d(in_channels, out_channels, stride=stride)
        self.bn1 = nn.BatchNorm3d(out_channels, momentum=bn_momentum, affine=(location != 3))
        self.conv2 = conv3d(out_channels, out_channels)
        self.bn2 = nn.BatchNorm3d(out_channels, momentum=bn_momentum)
        self.relu = nn.ReLU(inplace=True)
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                conv3d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm3d(out_channels, momentum=bn_momentum),
            )
        else:
            self.downsample = None
        # Film-specific variables
        self.location = location
        if self.location == 2 and self.downsample is None:
            raise ValueError("This is equivalent to location=1 and no downsampling!")
        # location decoding
        self.film_dims = 0
        if location in {0, 1, 2}:
            self.film_dims = in_channels
        elif location in {3, 4}:
            self.film_dims = out_channels
        if activation == "sigmoid":
            self.scale_activation = nn.Sigmoid()
        elif activation == "tanh":
            self.scale_activation = nn.Tanh()
        elif activation == "linear":
            self.scale_activation = None


    def rescale_features(self, feature_map, x_aux):
        """method to recalibrate feature map x"""

    def forward(self, feature_map, x_aux):

        if self.location == 0:
            feature_map = self.rescale_features(feature_map, x_aux)
        residual = feature_map

        if self.location == 1:
            residual = self.rescale_features(residual, x_aux)

        if self.location == 2:
            feature_map = self.rescale_features(feature_map, x_aux)
        out = self.conv1(feature_map)
        out = self.bn1(out)

        if self.location == 3:
            out = self.rescale_features(out, x_aux)
        out = self.relu(out)

        if self.location == 4:
            out = self.rescale_features(out, x_aux)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(residual)
        out += residual
        out = self.relu(out)

        return out


class DAFTBlock(FilmBase):
    # Block for ZeCatNet
    def __init__(
        self,
        in_channels: int=1,
        out_channels: int=3,
        bn_momentum: float = 0.1,
        stride: int = 2,
        ndim_non_img: int = 10,
        location: int = 3,
        activation: str = "linear",
        scale: bool = True,
        shift: bool = True,
        bottleneck_dim: int = 15,
    ) -> None:

        super().__init__(
            in_channels=in_channels,
            out_channels=out_channels,
            bn_momentum=bn_momentum,
            stride=stride,
            ndim_non_img=ndim_non_img,
            location=location,
            activation=activation,
            scale=scale,
            shift=shift,
        )

        self.bottleneck_dim = bottleneck_dim
        aux_input_dims = self.film_dims
        # shift and scale decoding
        self.split_size = 0
        if scale and shift:
            self.split_size = self.film_dims
            self.scale = None
            self.shift = None
            self.film_dims = 2 * self.film_dims
        elif not scale:
            self.scale = 1
            self.shift = None
        elif not shift:
            self.shift = 0
            self.scale = None

        # create aux net
        layers = [
            ("aux_base", nn.Linear(ndim_non_img + aux_input_dims, self.bottleneck_dim, bias=False)),
            ("aux_relu", nn.ReLU()),
            ("aux_out", nn.Linear(self.bottleneck_dim, self.film_dims, bias=False)),
        ]
        self.aux = nn.Sequential(OrderedDict(layers))

    def rescale_features(self, feature_map, x_aux):

        squeeze = self.global_pool(feature_map)
        squeeze = squeeze.view(squeeze.size(0), -1)
        squeeze = torch.cat((squeeze, x_aux), dim=1)

        attention = self.aux(squeeze)
        if self.scale == self.shift:
            v_scale, v_shift = torch.split(attention, self.split_size, dim=1)
            v_scale = v_scale.view(*v_scale.size(), 1, 1, 1).expand_as(feature_map)
            v_shift = v_shift.view(*v_shift.size(), 1, 1, 1).expand_as(feature_map)
            if self.scale_activation is not None:
                v_scale = self.scale_activation(v_scale)
        elif self.scale is None:
            v_scale = attention
            v_scale = v_scale.view(*v_scale.size(), 1, 1, 1).expand_as(feature_map)
            v_shift = self.shift
            if self.scale_activation is not None:
                v_scale = self.scale_activation(v_scale)
        elif self.shift is None:
            v_scale = self.scale
            v_shift = attention
            v_shift = v_shift.view(*v_shift.size(), 1, 1, 1).expand_as(feature_map)
        else:
            raise AssertionError(
                f"Sanity checking on scale and shift failed. Must be of type bool or None: {self.scale}, {self.shift}"
            )

        return (v_scale * feature_map) + v_shift


class Fusion2022(nn.Module):
    def __init__(self, in_channels=1, n_outputs=3, bn_momentum=0.1, n_basefilters=4, ndim_non_img=10) -> None:
        super().__init__()


        self.conv1 = ConvBnReLU(in_channels, n_basefilters, bn_momentum=bn_momentum)
        self.pool1 = nn.MaxPool3d(2, stride=2)  # 32
        self.block1 = ResBlock(n_basefilters, n_basefilters, bn_momentum=bn_momentum)
        self.block2 = ResBlock(n_basefilters, 2 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 16
        self.block3 = ResBlock(2 * n_basefilters, 4 * n_basefilters, bn_momentum=bn_momentum, stride=2)  # 8
        self.blockX = FBlock(4 * n_basefilters, 8 * n_basefilters, bn_momentum=bn_momentum)  # 4
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Linear(32, 16)
        self.fc1 = nn.Linear(16, n_outputs)

        layers = [
            ("aux_base", nn.Linear(ndim_non_img, 7, bias=False)),
            ("aux_relu", nn.ReLU()),
            # ("aux_dropout", nn.Dropout(p=0.2, inplace=True)),
            ("aux_1", nn.Linear(7, 4, bias=False)),
        ]

        layers2 = [
            ("aux_base", nn.Linear(ndim_non_img, 15, bias=False)),
            ("aux_relu", nn.ReLU()),
            # ("aux_dropout", nn.Dropout(p=0.2, inplace=True)),
            ("aux_1", nn.Linear(15, 32, bias=False)),
        ]
        self.aux = nn.Sequential(OrderedDict(layers))
        self.aux1 = nn.Sequential(OrderedDict(layers2))



    def forward(self, image, tabular):
        out = self.conv1(image)
        out = self.pool1(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.blockX(out, tabular)
        out = self.global_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)

        return out

#
class FBlock(nn.Module):
    def __init__(self, in_channels, out_channels, bn_momentum=0.05, stride=1, n_basefilters=4, ndim_non_img=10):
        super().__init__()
        self.conv1 = conv3d(in_channels, out_channels, stride=stride)
        self.bn1 = nn.BatchNorm3d(out_channels, momentum=bn_momentum)
        self.conv2 = conv3d(out_channels, out_channels)
        self.bn2 = nn.BatchNorm3d(out_channels, momentum=bn_momentum)
        self.relu = nn.ReLU(inplace=True)
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        self.global_pool1 = nn.AvgPool3d(5)
        self.fc5 = nn.Linear(32, 4)

        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                conv3d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm3d(out_channels, momentum=bn_momentum),
            )
        else:
            self.downsample = None

        layers = [
            ("aux_base", nn.Linear(ndim_non_img, 7, bias=False)),
            ("aux_relu", nn.ReLU()),
            # ("aux_dropout", nn.Dropout(p=0.2, inplace=True)),
            ("aux_1", nn.Linear(7, n_basefilters, bias=False)),
        ]
        self.aux = nn.Sequential(OrderedDict(layers))

    def forward(self, x, tabular):
        residual = x
        out = self.conv1(x)
        out2 = self.bn1(out)
        batch_size, n_channels = out2.size()[:2]
        attention = self.aux(tabular)
        cat2 = self.fc5(out2)
        out = torch.cat(attention, cat2)
        out = Attention(out)

        out2 = self.relu(out)
        out = self.conv2(out2)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


from sklearn.metrics import confusion_matrix
import numpy as np



def ACC(Y_test, Y_pred, n):
    acc = []
    con_mat = confusion_matrix(Y_test, Y_pred)
    for i in range(n):
        number = np.sum(con_mat[:, :])
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i, :]) - tp
        fp = np.sum(con_mat[:, i]) - tp
        tn = number - tp - fn - fp
        acc1 = (tp + tn) / number
        acc.append(acc1)

    return acc
import os



def creat_filelist(input_path, classes):

    dir_image1 = []
    file_list = []
    for index, name in enumerate(classes):
        print('index', index)
        index_str = str(index)
        dir_image1_temp = input_path + '/' + name + '/'
        for dir2 in os.listdir(dir_image1_temp):
            dir_image2_temp = dir_image1_temp + '/' + dir2 + ' ' + index_str
            # dir_image2_temp1 = dir_image2_temp.join(' ')
            # dir_image2_temp2 = dir_image2_temp.join(index)
            file_list.append(dir_image2_temp)

    return dir_image1, file_list


def creat_txtfile(output_path, file_list):
    with open(output_path, 'w') as f:
        for list in file_list:
            print(list)
            f.write(str(list) + '\n')


def main():
    dir_image0 = '/content/drive/My Drive/pythonProject9/datasetADNI0923/train'
    dir_image1 = os.listdir(dir_image0)
    classes = dir_image1
    print(classes)
    dir_list, file_list = creat_filelist(dir_image0, classes)

    # print(file_list[0:3])
    output_path = 'train.txt'
    creat_txtfile(output_path, file_list)
    # print(output_path)


if __name__ == '__main__':
    main()

from sklearn.metrics import confusion_matrix
import numpy as np

def specificity(Y_test, Y_pred, n):

    spe = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        number = np.sum(con_mat[:,:])
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        fp = np.sum(con_mat[:,i]) - tp
        tn = number - tp - fn - fp
        spe1 = tn/ (tn + fp)
        spe.append(spe1)
    return spe

import os
import random
import shutil
from shutil import rmtree
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the dataset root directory in Google Drive
data_root = "/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD" # or the actual path where your data is located.

# Function to create directories
def mk_file(file_path: str):
    if os.path.exists(file_path):
        rmtree(file_path)
    os.makedirs(file_path)

# Set random seed for reproducibility
random.seed(0)

# Define split ratio (20% for validation)
split_rate = 0.2

# Define the original dataset path inside Google Drive
# This is the line that needs to be changed! Point it to your actual dataset directory
origin_data_path = data_root # or the actual path to your 'content' folder. If your data is directly under 'data_root', then just use 'data_root'.

# Ensure the dataset exists
assert os.path.exists(origin_data_path), "Dataset path not found!"

# Get class names (subfolders inside the dataset directory)
classes = [cls for cls in os.listdir(origin_data_path) if os.path.isdir(os.path.join(origin_data_path, cls))]

# Create train and validation directories
train_root = os.path.join(data_root, "train")
val_root = os.path.join(data_root, "val")

mk_file(train_root)
mk_file(val_root)

# Create class subdirectories inside train and val folders
for cls in classes:
    mk_file(os.path.join(train_root, cls))
    mk_file(os.path.join(val_root, cls))

# Split images into train and validation sets
for cls in classes:
    cls_path = os.path.join(origin_data_path, cls)
    images = os.listdir(cls_path)
    num_images = len(images)

    # Select random images for validation set
    val_images = random.sample(images, k=int(num_images * split_rate))

    for index, image in enumerate(images):
        src_path = os.path.join(cls_path, image)

        if image in val_images:
            dest_path = os.path.join(val_root, cls)
        else:
            dest_path = os.path.join(train_root, cls)

        shutil.move(src_path, dest_path)
        print(f"\r[{cls}] Processing [{index + 1}/{num_images}]", end="")

    print()  # Newline after processing each class

print("Processing done!")




Streaming output truncated to the last 5000 lines.
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (103).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (145).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (102).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (135).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (147).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (141).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (134).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (140).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (126).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (137).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (138).jpg 0
/content/drive/My Drive/pythonProject9/datasetADNI0923/train/AD//32 (